# A complete Hipfort application

In the <a href="../L1_Fortran_Refresher/Fortran.html">Fortran Refresher lesson</a> <a href="../L1_Fortran_Refresher/Fortran.ipynb">(ipynb)</a> we covered the essentials of the Fortran language and how to use `subroutines`, `functions`, `pointers`, `modules` as well as how to call C code from Fortran. If these topics are unfamiliar to you, then it might be useful to review the material in that section first!

From the <a href="../L2_GPU_Computing_Fundamentals/Fundamentals.html">GPU Computing Fundamentals </a><a href="../L2_GPU_Computing_Fundamentals/Fundamentals.ipynb">(ipynb)</a> section, every accelerated application has the same basic design:

1. At program launch, compute devices are discovered and initialized.
2. Memory spaces are allocated on the compute device.
3. Kernels are prepared.
4. Memory is copied from the host to the compute device.
5. Kernels are run to perform whatever compute operation is required.
6. The output from kernel runs is copied back from the compute device to the host. IO may then occur before the next iteration.
  
**Steps 4-6** are repeated as many times as neccessary until the program is done, then at completion of the program

7. Deallocate memory, 
8. Release resources and exit.

## Tensor addition math

In this section we are going to walk through each of these steps as part of a complete example with hipFORT, using 2D tensor addition as the basic algorithm. For 2D tensors **A**, **B**, and **C**, each of size (M,N), the following relationship holds true at each index (i,j) in the tensors.

$$
A(i,j)+B(i,j)=C(i,j)
$$

In the prior **Fortran Refresher** section we used CPU code in Fortran and C to compute the answer $C(i)$ for 1D tensor addition. In this example we are going to use a HIP Kernel on the GPU to compute the answer $C(i,j)$ at every location in **C**.


## Example applications

In HIP we need a way to get a handle on the memory allocations that are on the compute device. HipFORT can use either a `C` pointer of Fortran type `c_ptr` or a Fortran pointer as a handle to the memory allocations on the GPU. The methods of working with each type are subtly different though. In the applications 

* [tensoradd_hip_cptr.f90](tensoradd_hip_cptr.f90)
* [tensoradd_hip_fptr.f90](tensoradd_hip_fptr.f90)

we use C pointers and Fortran pointers to perform 2D tensor addition. It will be helpful to have **both files open** at the same time for comparison.

## Use the hipFORT API

Access to all hipFORT functions is via the `hipfort` and `hipfort_check` modules. We bring those modules in along with others at the beginning of the program.

```Fortran
    ! HIP modules
    use hipfort
    use hipfort_check
```

## Check HIP API calls

HipFORT functions usually have a **return type** that we can check to make sure everything worked ok. If these checks are **not performed** some functions will return even though there has been a **silent failure**.  It is therefore **best practice** to **always** the check the return type from hipFORT calls. The `hipfort_check` module defines a subroutine called `hipcheck` that we can use to wrap around a hipFORT API call. It then checks the return type and exits the program if there has been an error. For example we wrap `hipcheck` around a `hipmalloc` call as follows:

```
call hipcheck(hipmalloc(A_d, M, N))
```

## Code validation

It is important to make sure that the output of the compuation is accurate for every element in the output. Wrong answers can be computed very quickly but they are of no value! In the **src** folder is a file called <a href="../src/math_utils.f90">math_utils.f90</a>. It contains a module called `math_utils` that is intended to contain functions for working with tensors on the CPU. Within the module is a function called `check_tensor_addition_2D` that iterates over every point in a passed-in tensor $C(i,j)$ and checks to see each point is within an error margin of $A(i,j)+B(i,j)$. The function has the following signature, where **A**, **B**, and **C** are arrays on the host:

```Fortran
function check_tensor_addition_2D(A, B, C, eps_mult) result(success)
            !! Function to check the outcome of tensor addition
            !! only check the host arrays

            real(kind=c_float), dimension(:,:), intent(in), pointer :: A, B, C
        
            real, intent(in) :: eps_mult
                !! Epsilon multiplier, how many floating point spacings
                !! can the computed answer be from our benchmark answer
```

Within the tensoradd programs we import `check_tensor_addition_2D` and rename it as the function `check` 

```Fortran
! Maths check
    use math_utils, only : check => check_tensor_addition_2D
```

## Fortran interface to kernel launch function

HipFORT doesn't yet have a way to launch kernels, however passing C-pointers from Fortran to C/C++ functions is straightforward, and from C/C++ code we can launch kernels. In the file [kernel_code.cpp](kernel_code.cpp) is a C  function called `launch_kernel_hip` that does the job of launching kernels. It has the following signature:

```Fortran
    void launch_kernel_hip(
            float_type* A, 
            float_type* B,
            float_type* C,
            int M,
            int N) {
```

In order to call this function from Fortran we define an `interface` to the function within the programs of [tensoradd_hip_cptr.f90](tensoradd_hip_cptr.f90) and [tensoradd_hip_fptr.f90](tensoradd_hip_fptr.f90) as follows:

```Fortran
    interface
        ! A C function with void return type
        ! is regarded as a subroutine in Fortran 
        subroutine launch_kernel_hip(A, B, C, M, N) bind(C)
            use iso_c_binding
            ! Fortran passes arguments by reference as the default
            ! Arguments must have the "value" option present to pass by value
            ! Otherwise launch_kernel will receive pointers of type void**
            ! instead of void*
            type(c_ptr), intent(in), value :: A, B, C
            integer(c_int), intent(in), value :: M, N
        end subroutine
        
    end interface

```

Note the presence of the `value` option for the input arguments. This is so we pass arguments by `value` instead of by **reference** (the default). If we didn't have the value keyword the C function would receive a reference (or pointer to the variables) instead of a copy of the variables. In the case of `launch_kernel_hip` without the `value` keyword in the interface then A would be of type `void**` instead of `void*`.

## Strategies for maintaining consistency in precision

In the file [kernel_code.cpp](kernel_code.cpp) you might have noticed the use of `float_type`. This is a typedef that determines the data type of elements within the tensors. It is set to `double` in <a href="../src/kinds.h">../src/kinds.h</a> if the macro `USE_C_DOUBLE` is defined. Otherwise it is type-defined as `float`.

```C++
#ifdef USE_C_DOUBLE
    typedef float float_type;
#else
    typedef double float_type;
#endif
```

We include `kinds.h` from [kernel_code.cpp](kernel_code.cpp) to make this type available. 

```C++
// Include this to make float_type available
#include "kinds.h"
```

Element types must **be consistent** when arrays are accessed from both Fortran and C/C++. In Fortran the `iso_c_binding` module provides  `c_float` and `c_double` kinds that are equivalent to `float` and `double` in C/C++.  The `iso_fortran_env` module has `real32` and `real64` kinds for creating reals of 32-bit and 64-bit floats. These correspond to the C types `float` and `double` for IEEE-754 compliant floating point implementations. 

In <a href="../src/kinds.f90">../src/kinds.f90</a> we employ a similar method to declare `float_type` as a kind within the `kinds` module. We use associate `float_type` with `c_double` if `USE_C_DOUBLE` is defined, and with `c_float` otherwise.

```Fortran
#ifdef USE_C_DOUBLE
   ! Set float_type as c_double from iso_c_binding module
   use, intrinsic :: iso_c_binding, only : float_type => c_double
#else
   ! Set float_type as c_float from iso_c_binding module
   use, intrinsic :: iso_c_binding, only : float_type => c_float
#endif
```

In [tensoradd_hip_cptr.f90](tensoradd_hip_cptr.f90) we can `use` the `kinds` module and create real variables of kind `float_type`.

```Fortran
! Use the kinds module to make available the float_type kind
    use kinds

    ...

    ! Fortran pointers to memory allocations on the host
    real(float_type), dimension(:,:), pointer :: A_h, B_h, C_h 
```

Now we can be sure that `float_type` corresponds to a floating point number of a consistent precision, even when compiler flags set the default precision of `reals` to something else. One less source of bugs!

## Select and manage a HIP device

Every HIP device that a program has access to is associated with a `primary context`. The primary context is a resource manager for keeping track of all the resources allocated on that device for the program. Host threads share access to primary contexts in a way that is (or at least is intended to be!) thread safe. Every host thread in an program is **free to choose** which device to use. Usually the HIP runtime is initialised, a primary context is created and a host thread is **connected** to the first available device (device 0) whenever that host thread makes its first call to a HIP function. For environments and programs where there are multiple devices and host threads, it is **good practice** to explicity initialize the HIP API and be specific about which device you would like the host thread to connect to. In the file <a href="../src/hip_utils.f90">../src/hip_utils.f90</a> are two subroutines `init_device` and `reset_device` that provide a way to choose a GPU device and reset (release all resources) in the selected device's primary context. The first statement after variable declarations in  [tensoradd_hip_cptr.f90](tensoradd_hip_cptr.f90) and [tensoradd_hip_fptr.f90](tensoradd_hip_fptr.f90) is to initialize HIP and choose the GPU.

```Fortran
    ! Find and set the GPU device. Use device 0 by default
    call init_device(0)   
```

The argument to `init_device` is the desired index of the device that we'd like to use. Device indices start at 0 and in this instance we select the first available gpu (with id 0). Inside the subroutine `init_device` we initialize the HIP API using a call to `hipinit`.

```Fortran
call hipcheck(hipinit(0))
```

The call to `hipinit` only needs to be done once, so we have a variable `acquired` within the module to make sure of this. 

Within `init_device`, we then call `hipgetdevicecount` to poll the number of valid devices. If the desired device index (the input argument to `init_device`) falls within the range of valid device then we call `hipsetdevice` to set the HIP device according to the desired device index. Any subsequent HIP calls from a host thread will then use the selected GPU.

```Fortran
 ! Get the number of compute devices
        call hipcheck(hipgetdevicecount(ndevices))
            
        if ((dev_id .ge. 0) .and. (dev_id .lt. ndevices)) then
            ! Choose a compute device
            call hipcheck(hipsetdevice(dev_id))
        else
            write(error_unit,*) 'Error, dev_id was not inside the range of available devices.'
            stop 1
        end if
```

The function `reset_device` in [hip_utils.f90](hip_utils.f90) calls `hipdevicesynchronize` to make sure the selected GPU device is finished with all pending activity, then it calls `hipdevicereset` to release all resources in the primary context. 

```Fortran
        ! Release all resources on the gpu
        if (acquired) then
            ! Make sure the GPU is finished
            ! with all pending activity
            call hipcheck(hipdevicesynchronize())

            ! Now free all resources on the primary context
            ! of the selected GPU
            call hipcheck(hipdevicereset())
        end if
```

It is **best practice** to reset the compute device at the end of the computation, but make sure that no other threads are using resources on that GPU when you do it!

## Memory on the device

### Standard data types on the host

Next, we allocate memory for the tensors on both the host and the compute device. Fortran has the ability to change, with a compiler flag, how many bytes are used for `real` and `integer` types. When we work with Fortan and C/C++ precision needs to be consistent. We use the `float_type` discussed earlier to declare pointers `A_h`, `B_h`, and `C_h` for arrays on the host.

```Fortran
real(float_type), dimension(:,:), pointer :: A_h, B_h, C_h

! Allocate memory on host 
allocate(A_h(M,N), B_h(M, N), C_h(M,N))
```

### Variable naming convention

Notice the `_h` suffix on variable names. In this module we choose to put a `_h` suffix on memory allocations that reside on the host and a `_d` suffix for memory allocations that reside on the compute device. It is a variable naming convention that makes it easier to see what memory is allocated where.

### C pointers and Fortran pointers

Both C pointers of type `c_ptr` and Fortran pointers can be used as handles to memory allocations on the compute device. C pointers are flexible but not very safe. Fortran pointers are also not very safe but additionally  retain information on the shape, data type, and size of the allocation.

In [tensoradd_hip_cptr.f90](tensoradd_hip_cptr.f90) we use C pointers for memory allocations to tensors **A**, **B**, and **C** on the compute device

```Fortran
    ! C Pointers to memory allocations on the device
    type(c_ptr) :: A_d, B_d, C_d
```

and in [tensoradd_hip_fptr.f90](tensoradd_hip_fptr.f90) we use Fortran pointers.

```Fortran
    ! Fortran pointers to memory allocations on the device
    real(float_type), dimension(:,:), pointer :: A_d, B_d, C_d
```

### Allocate device memory

The `hipMalloc` function allocates memory in the **global** memory space on the compute device. This memory is the largest (and slowest) memory on the compute device. Memory allocated with `hipMalloc` is accessible from every kernel that runs on the compute device but not from the host. Either C pointers or Fortran pointers may be used with `hipMalloc`.

When using hipmalloc with **C pointers** we need to specify how many **bytes** to reserve. The `sizeof` function returns the number of bytes allocated for a Fortran pointer. In [tensoradd_hip_cptr.f90](tensoradd_hip_cptr.f90) we use the bytes allocated for host arrays as an input argument when allocating **A_d**, **B_d**, and **C_d**. 

```Fortran
    ! Allocate tensors on the device
    call hipcheck(hipMalloc(A_d, int(sizeof(A_h), c_size_t)))
    call hipcheck(hipMalloc(B_d, int(sizeof(B_h), c_size_t)))
    call hipcheck(hipMalloc(C_d, int(sizeof(C_h), c_size_t)))
```

The `sizeof` intrinsic function produces a different data type across different compilers, therefore we the `int` function to make sure the number of bytes returned is an integer of kind `c_size_t`.

Memory allocations that use Fortran pointers need **elements** (not bytes) as the input argument for allocation with `hipMalloc`. In [tensoradd_hip_fptr.f90](tensoradd_hip_fptr.f90) we specify the size of the arrays to allocate in elements along each dimension.

```Fortran
    ! Allocate memory on the device
    call hipcheck(hipMalloc(A_d, M, N))
    call hipcheck(hipMalloc(B_d, M, N))
    call hipcheck(hipMalloc(C_d, M, N))
```

There are additional ways to allocate memory with Fortran pointers. For example we could have used the `hipmalloc_r4_c_size_t` function to allocate the 2D arrays, each element using 4 bytes, and having integer variables of kind `c_size_t` to specify dimensions.


```Fortran
    ! Could have also done this for the allocate instead
    call hipcheck(hipMalloc_r4_2_c_size_t(A_d, int(M_in, c_size_t), int(N_in, c_size_t)))
    call hipcheck(hipMalloc_r4_2_c_size_t(B_d, int(M_in, c_size_t), int(N_in, c_size_t)))
    call hipcheck(hipMalloc_r4_2_c_size_t(C_d, int(M_in, c_size_t), int(N_in, c_size_t)))
```

It is **important** to note that while Fortran pointers can be used as a **handle** on GPU memory allocations, they can't actually be used to access the GPU allocation from within Fortran code on the host. For example this would result in a memory access violation.

```Fortran
A_d(1,1) = 1.0
```

The reason is that when allocating with `hipMalloc`, the actual allocation is part of the **memory space on the GPU** and **not on the host**. Using Fortran pointers to access memory allocated with `hipMallocManaged` **is permissible**, because managed memory permits access from both GPU and the host.

### De-allocate device memory

When device memory is no longer needed, the **hipFree** function deallocates device memory with both C and Fortran pointers. We do this at the end of the program.

```Fortran
    ! Free allocations on the GPU
    call hipcheck(hipFree(A_d))
    call hipcheck(hipFree(B_d))
    call hipcheck(hipFree(C_d))
```

It is **best practice** to make sure pointers are set to null when they no longer point to something. For Fortran pointers ([tensoradd_hip_fptr.f90](tensoradd_hip_fptr.f90)) we use the `nullify` function

```Fortran
    ! It is best practice to nullify all pointers 
    ! once we are done with them 
    nullify(A_h, B_h, C_h, A_d, B_d, C_d)
```

and for C pointers ([tensoradd_hip_cptr.f90](tensoradd_hip_cptr.f90)) we set them to `c_null_ptr`.

```Fortran
    ! Set C pointers to null as well
    A_d = c_null_ptr
    B_d = c_null_ptr
    C_d = c_null_ptr
```

## Memory copies between host and device

Memory can be copied between host and device allocations, or between device allocations. After filling arrays **A_h** and **B_h** we proceed to copy them to the device allocations **A_d** and **B_d**.

### Copy from host to device

The `hipMemcpy` function can use either C pointers or Fortran pointers. Here is the code to copy from host to device using C pointers.

```Fortran
    ! Copy memory from the host to the device 
    call hipcheck(hipMemcpy(A_d, c_loc(A_h), int(sizeof(A_h), c_size_t), hipMemcpyHostToDevice))
    call hipcheck(hipMemcpy(B_d, c_loc(B_h), int(sizeof(B_h), c_size_t), hipMemcpyHostToDevice))
```

Each `hipmemcpy` call has a additional flag to specify the direction of the copy. There are five options available:

* `hipMemcpyHostToHost`
* `hipMemcpyHostToDevice`
* `hipMemcpyDeviceToHost`
* `hipMemcpyDeviceToDevice`
* `hipMemcpyDefault`

The `hipMemcpyDefault` option tries to infer the direction of transfer from the memory spaces of the input pointers. It is less readable however.

Hipmemcpy also works with Fortran pointers, though when specifying the size to copy we specify **elements** instead of **bytes**! Notice the use of `size` instead of `sizeof` to specify elements instead of bytes.

```Fortran
    call hipcheck(hipMemcpy(A_d, A_h, size(A_h), hipMemcpyHostToDevice))
    call hipcheck(hipMemcpy(B_d, B_h, size(B_h), hipMemcpyHostToDevice))
```

In the case of Fortran pointers we could have also used `hipMemcpy` functions that are specific to the arrays in question, for example we could also have done this.

```Fortran
    ! Could also have done this for the copy instead
    !call hipcheck(hipMemcpy_r4_2_c_size_t(A_d, A_h, &
    !    int(size(A_h), c_size_t), hipMemcpyHostToDevice))
    !call hipcheck(hipMemcpy_r4_2_c_size_t(B_d, B_h, &
    !    int(size(B_h), c_size_t), hipMemcpyHostToDevice))
```

### Copy from device to host

After running the kernel, we copy **C_d** back to **C_h**, using either C pointers,

```Fortran
    ! Copy memory from the device to the host
    call hipcheck(hipMemcpy(c_loc(C_h), C_d, sizeof(C_h), hipMemcpyDeviceToHost))
```
or Fortran pointers

```Fortran
    ! Copy from the device result back to the host
    call hipcheck(hipMemcpy(C_h, C_d, size(C_d), hipMemcpyDeviceToHost))
```

## Kernel source and launch

### Call the kernel launch function

Since the Hipfort API doesn't have the functionality to define and launch kernels, we use the C function `launch_kernel_hip` to launch the kernel. This function has as the input argument C pointers for **A**, **B**, and **C** on the device and the integers **M** and **N** for the array sizes. In [tensoradd_hip_cptr.f90](tensoradd_hip_cptr.f90) we can just use the pointers **A_d**, **B_d**, and **C_d** directly while taking special care to convert the integer arguments to the type required by the function.

```Fortran
    ! Call the C function that launches the kernel
    call launch_kernel_hip( &
        A_d, &
        B_d, &
        C_d, &
        int(M, c_int), &
        int(N, c_int) &
    )
```

In [tensoradd_hip_fptr.f90](tensoradd_hip_fptr.f90) we must use `c_loc` to get the C pointer that underlies the Fortran pointers.

```Fortran
    ! Call the C function that launches the kernel
    call launch_kernel_hip( &
        c_loc(A_d), &
        c_loc(B_d), &
        c_loc(C_d), &
        int(M, c_int), &
        int(N, c_int) &
    )
```

### Kernel launch function

Let's examine the file [kernel_code.cpp](kernel_code.cpp). 

#### C linkage

The kernel launch function `launch_kernel_hip` is wrapped in an `extern "C"` code block to ensure the function is compiled with C linkage. This mean it's name doesn't get mangled during compilation and is therefore accessible from Fortran.

```C++
// C function to call the tensoradd_2D kernel
extern "C" {

    void launch_kernel_hip(
            float_type* A, 
            float_type* B,
            float_type* C,
            int M,
            int N) {
```

#### Role of the kernel launch function

From the **GPU Computing Fundamentals** section we have the following diagram of a Grid that is made up of Blocks.

<figure style="margin: 1em; margin-left:auto; margin-right:auto; width:90%;">
    <img src="../images/Grid.svg">
    <figcaption style= "text-align:lower; margin:1em; float:bottom; vertical-align:bottom;">A Grid in the context of GPU computing. Grids are made up of Blocks and Blocks are made up of Threads</figcaption>
</figure>

It is the job of the kernel launch function to: 

* Pass arguments to the kernel 
* Determine the block size, (number of threads along each dimension of the block)
* Determine the grid size, (number of blocks along each dimension of the grid)
* Launch the kernel and examine launch errors
* Optionally synchronize the device

#### Define block size and grid size

The `dim3` structure (with fields `x`, `y`, `z`) is used to specify the block size and the number of blocks per dimension.

```C++       
        // Grid size
        dim3 global_size = {
            (uint32_t)(M), 
            (uint32_t)(N)
        }; 
        
        // Block size, 
        dim3 block_size = {8,8,1};
        
        // Number of blocks in each dimension
        dim3 nblocks = {
            global_size.x/block_size.x,
            global_size.y/block_size.y,
            1
        };
```

We must always have an integer number of blocks along every dimension of the grid. Sometimes this means making a grid that is larger than we need. This is fine provided we **build memory access protection into the kernel** so we don't run off the end of the arrays. 
    
```C++
        // Make sure there are enough blocks
        if (global_size.x % block_size.x) nblocks.x += 1;
        if (global_size.y % block_size.y) nblocks.y += 1;
        if (global_size.z % block_size.z) nblocks.z += 1;
```

#### Shared memory

HIP provides the ability to define a small amount of **shared memory** that is available to all threads in a block. This memory is fast and can be used as a small scratch space. We don't need shared memory for this example so we specify `0` as the number of bytes to allocate for shared memory.


```C++
        // Decide on the number of bytes to allocate for shared memory
        size_t sharedMemBytes = 0;
```

#### Kernel launch with hipLaunchKernelGGL

Finally we get to launch the kernel itself. There are a few ways to do this, here we use the **hipLaunchKernelGGL** macro to launch the kernel `tensoradd_2D` with the specified block and grid size along with kernel arguments. A `stream` in HIP can be thought of as a work queue to which we submit work, we use stream 0 which is the default or null stream.

```C++
        // Launch the kernel
        hipLaunchKernelGGL(
                // Kernel name
                tensoradd_2D,
                // Number of blocks per dimension
                nblocks,
                // Number of threads along each dimension of the block
                block_size,
                // Number of bytes dynamically allocated for shared memory
                sharedMemBytes,
                // Stream to use (0 is the default or null stream)
                0,
                // Kernel arguments
                A, B, C,
                M, N);
```

#### Kernel launch with CUDA syntax

One can also use the CUDA-like triple-chevron syntax to launch a kernel. This is not ANSI C++ compliant, however it isn't much of a problem because only compilers that understand triple chevrons (hipcc, nvcc) will be used to compile this source file.

```C++
        // The triple-chevron (non C++ compliant) way of launching kernels
        tensoradd_2d<<<nblocks, block_size, sharedMemBytes, 0>>>(A, B, C, M, N);
```

#### Check kernel launch

We use the `hipGetLastError` function to see if there were any problems arising from kernel launch. The macro `HIPCHECK` is defined earlier in the file [kernel_code.cpp](kernel_code.cpp) and behaves similarly to the Fortran subroutine `hipcheck` defined in `hipfort_check`.

```C++
        // Make sure the kernel launch went ok
        HIPCHECK(hipGetLastError());
```

#### Synchronize the compute device

Finally, we use the `hipDeviceSynchronize` function to make sure that the kernel is finished before continuing. This step is not strictly necessary because the subsequent copy of **C_d** to **C_h** will use the same (null) stream and will block until the kernel is finished. 

```C++
    	// Wait for the kernel to finish
    	HIPCHECK(hipDeviceSynchronize()); 
    }
}
```

### Kernel source 

Let's examine the source of the kernel we are launching to perform 2D tensor addition.  Notice that kernel code is always a function with a return type of `void` and has the `__global__` qualifier. This qualifier means the function can be launched from the host and will run on the device. Other qualifiers such as `__host__` or `__device__` permit the function to also be used on the host or called from a kernel on the device.

```C++
// Kernel to perform 2D tensor addition
__global__ void tensoradd_2D (
	    // Memory allocations
        float_type* A, 
        float_type* B, 
        float_type* C,
        // Size of the problem
        int M,
        int N) {

    // Any dynamically allocated memory is available here
    extern __shared__ float_type shared[];

    // We adopt column-major indexing for this example
    
    // Compute (zero-based) indicies within grid
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    // Due to block sizes 
    // i and j might lie outside M and N
    // make sure we don't run off the domain
    // of the grid
    if ((i<M) && (j<N)) {
        // 1D position within 2D arrays
        // stride down a column is 1
        // stride along a row is M
        
        size_t offset = i + j*M;

        // Now perform the 2D tensor addition
        C[offset] = A[offset] + B[offset];
    }
}
``` 

#### Kernel arguments

We define pointers **A**, **B**, **C** to memory of `float_type` as inputs to the kernel. These are pointers to memory allocations in the **global** memory space on the compute device. In the host program we pass in the pointers **A_d**, **B_d**, **C_d** for use in the kernel as **A**, **B**, **C**. We also pass in the integers **M** and **N** that represent the size of the 2D tensors.

```C++
// Kernel to perform 2D tensor addition
__global__ void tensoradd_2D (
	    // Memory allocations
        float_type* A, 
        float_type* B, 
        float_type* C,
        // Size of the problem
        int M,
        int N) {
```

#### Shared memory

Any shared memory that we specified in the kernel launch function is available through this line of code where we make shared memory available as a pointer to an array of type `char`. 

```C++
    // Any dynamically allocated memory is available here
    extern __shared__ char shared[];
```

We can only have one line of code that points to `extern __shared__` but we can declare pointers of any type that points to any location within this space. For example we could define a pointer called `shared_A` of type `float_type*` that points to the first element of the shared memory, like this.

```C++
    // Can use shared memory like this
    float_type* shared_A = (float_type*)&shared[0];
```

#### Locate the kernel within the grid

Within kernel code there are structures that can help us locate the kernel's position within the Grid at launch. Each structure has fields `{x, y, z}` that each contain a value for its index or length along the corresponding dimension in the grid.

| Structure | Explanation |
| --- | --- |
| gridDim | Number of blocks along each dimension of the grid. |
| blockIdx | Index of a block along a dimension of the grid. |
| blockDim | Number of threads along a dimension of a block. |
| threadIdx | Index of a thread along a dimension of the block. |

We use these structures to find the `(i, j)` coordinates of the kernel within the grid.

```C++
    // Compute (zero-based) indicies within grid
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
```

#### Kernel math and array indexing

At last! In the last lines of the kernel we can perform the actual 2D vector addition. Since the grid might be larger than the size of the 2D arrays, then some indices `(i,j)` may lie **outside** the bounds of **M** and **N**, depending on the choice of the block size. Therefore we enclose the kernel code within a protective `if` statement.

```C++
    if ((i<M) && (j<N)) {
        // 1D position within 2D arrays
        // stride down a column is 1
        // stride along a row is M
        
        size_t offset = i + j*M;

        // Now perform the 2D tensor addition
        C[offset] = A[offset] + B[offset];
    }
```

From the **GPU Computing Fundamentals** section, the position (offset) in a multi-dimensional allocation is given by $p=C \cdot S$, where $C$ is the coordinates and $S$ is the stride vector. The tensors are of size $(M,N)$, and we are using column-major indexing, therefore the stride vector is $(1, M)$. Given the coordinates are $C=(i,j)$ the `offset` is $(i,j) \cdot (1, M)= i \times 1 + j \times M = i + j \times M$. 

```C++
    size_t offset = i + j*M;
```

We use this offset to perform 2D tensor addition. Note that in C/C++ kernel code we use the square operator `[]` to access memory relative to a pointer, and indices are 0-based.

```C++
    // Now perform the 2D tensor addition
    C[offset] = A[offset] + B[offset];    
```

## Code validation

In <a href="../src/math_utils.f90">../src/math_utils.f90</a> we have imported the checking function `check_tensor_addition_2D` as `check`. We call this function to see if the computation was valid. The function will print an error and exit if the result in `C_h(i,j)` is not within `eps_mult` floating point spacings of `A_h(i,j)+B_h(i,j)`.

```Fortran
    ! Check the answer
    success = check(A_h, B_h, C_h, eps_mult)
```

## Resource cleanup

We have already covered the importance of using `hipFree` to deallocate any memory allocations created with `hipMalloc` followed by making Fortan pointers safe through using either `nullify`, and C pointers safe by setting to c_null_ptr. We finish with a call `reset_device` to make sure the GPU is finished work, and that all allocations have been cleaned up and the primary context is in a fresh state.

```Fortran
    ! Make sure all resources on the selected device are released
    call reset_device
```

## Memory safety with device allocations

As you might have noticed there are **numerous** opportunities for memory errors to arise with this code. For example, bugs can creep in if we:

* Don't check the return result on all API calls. This can result in **silent failures**.
* Don't specify the correct number of bytes or elements for memory allocations.
* Don't specify the correct number of bytes or elements when copying memory.
* Don't use guard clauses, or correct indexing math in kernels, and then try to use memory beyond device allocations. 
* Are not consistent with precision when passing data from Fortran to C, and from C to the kernel. This is a source of bugs that are **really hard** to diagnose.
* Don't initialize or copy memory when we should. Reading from allocated but uninitialized memory can result in extreme values in the output!
* Forget to deallocate pointers. This is a **memory leak**!
* Try to access memory through a Fortran pointer when the underling allocation is on the GPU.
* Forget to set pointers to null and then try to use them. This is called a **dangling pointer**.

### Memory safety issues with C pointers

The C pointer method is quite powerful in that the `c_loc` function can produce C pointers from all kinds of Fortran pointers and arrays with the `target` attribute. However with C pointers the size and data type information are decoupled from the pointer, leaving it to the programmer to make sure this information is paired with the pointer in any function calls.

### Memory safety issues with Fortran pointers

Fortran pointers are safer in that the size and data type of the allocation is encoded into the pointer as well as information on wether or not it is associated (pointing at a memory allocation). This provides opportunities for additional consistency checks when working with memory. As with C pointers, Fortran pointers are still vulnerable to memory leaks. 

## Use modern Fortran features for additional memory safety

The introduction of the Fortran 90 standard brought with it some object oriented features such as `derived types`, which are like C `structs` and have many object-oriented features seen in C++ classes, such type-bound procedures (subroutines that operate on internal data) and a `final` subroutine (destructor) that is called when an instance of the derived type goes out of scope. Types also can have `generic procedures` which are mappings so that one function name can map to many different functions, depending on the input data type. We can use derived types to build **additional safety** into working with memory allocations on a compute device. 

### Fortran types (classes)

In the file [tensor_hip.f90](tensor_hip.f90) is a `tensor` type whose memory allocation is on the compute device. It contains just three fields, a flag `allocd` to keep track of wether or not the memory is allocated, a C pointer named `mem` to contain the allocation, and an integer `nbytes` of kind `c_size_t` to keep track of the number of bytes allocated.

```Fortran 
    type :: tensor
        !! Object to represent a tensor allocated on the GPU

        ! Is this tensor allocated?
        logical :: allocd = .false.
        
        ! Pointer to the memory
        type(c_ptr) :: mem = c_null_ptr
        
        ! Number of bytes in the allocation
        integer(c_size_t) :: nbytes = 0
```

Following the `contains` clause we can define procedures that on internal data of the type. These procedures are just subroutines that work on the internal data of the type.

```Fortran    
        contains
        
            ! Upload procedures
            procedure :: copy_from_host_cptr
            procedure :: copy_from_host_float_type_1
            procedure :: copy_from_host_float_type_2
            
            ! Download procedures
            procedure :: copy_to_host_cptr
            procedure :: copy_to_host_float_type_1
            procedure :: copy_to_host_float_type_2
            
            ! Allocation and de-allocation procedures
            procedure :: malloc
            procedure :: free
```

Generic procedures are a way to provide polymorphism, a single procedure that can map to many procedures depending on the input arguments. We define two generic procedures `copy_from` and `copy_to` that copy arrays from the host to the tensor and from the tensor to the host for different types of input arguments. These map to the `copy_to` and `copy_from` procedures defined above.

```Fortran
! Generic procedures for different types of data
            generic :: copy_from => copy_from_host_cptr, &
                copy_from_host_float_type_1, &
                copy_from_host_float_type_2 !, can specify more comma-separated functions here
            generic :: copy_to => copy_to_host_cptr, &
                copy_to_host_float_type_1, &
                copy_to_host_float_type_2 !, can specify more comma-separated functions here
```

A `final` procedure is called when an instance of this type goes out of scope. It functions as a **destructor** for the type. We can use it to make sure the memory allocation is always cleaned up when the object is destroyed.

```Fortran    
            ! Final is a cleanup function when the object goes out of scope
            final :: destructor
            
end type tensor
```

#### Procedures

Within the [tensor_hip.f90](tensor_hip.f90) module are the subroutines and functions that the procedures refer to. Since the allocation status and number of bytes is defined within the type, this gives us opportunities to perform additional memory safety checks. For example the `malloc` procedure is defined as follows:

```Fortran
    ! Functions for the tensor class
    subroutine malloc(this, nbytes)
        !! Allocate memory for a tensor on the GPU
        
        ! Import the HIP modules
        use hipfort
        use hipfort_check

        ! Polymorphic variable for the class
        class(tensor), intent(inout) :: this

        ! Number of bytes to allocate
        integer(c_size_t), intent(in) :: nbytes

        ! Check to make sure we are not already allocated
        if (this%allocd) then
            call this%free
        end if

        ! Now allocate memory for the tensor on the GPU
        call hipCheck(hipMalloc(this%mem, nbytes))

        ! Set the allocated flag
        this%allocd = .true.

        ! Set the number of bytes in the allocation
        this%nbytes = nbytes
        
    end subroutine malloc
```

In a similar way to Python class member functions, procedures of a type in Fortran always have at least one argument called `this` which is of `class(tensor)`. The `class(tensor)` type is a polymorphic placeholder for both the `tensor` type and any types that inherit from it.

#### Access to type members and procedures

Access to any members and procedures of the type is through the `%` operator. For example within the `malloc` procedure we check the `allocd` variable and run the `free` procedure as follows:

```Fortran
    ! Check to make sure we are not already allocated
    if (this%allocd) then
        call this%free
    end if
```

#### Final procedure

The `final` procedure (which we name as `destructor`) needs to have `this` as an argument of `type(tensor)` to signify that it **must** be specific instance of the `tensor` class. In this subroutine we just call the `free` procedure to free memory when a tensor goes out of scope. This is a way to ensure that the memory allocation on the compute device is always released on exit.

```Fortran
subroutine destructor(this)
    !! Destructor, `this` must be of type(tensor) because it is valid only for instances
    !! of this type
    type(tensor), intent(inout) :: this
    call this%free
end subroutine destructor
```

### Using the tensor type

In the source file [tensoradd_hip_oo.f90](tensoradd_hip_oo.f90) we use the `tensor` type to work with memory allocations on the GPU in a way that has enhanced memory safety and fewer chances of bugs being introduced. 

#### Import the tensor type

We bring in the tensor type and rename it to `tensor_gpu` with this statement.

```Fortran
    ! Use the tensor type defined in tensor_hip.f90
    use tensor_hip, only : tensor_gpu => tensor
```

#### Define objects

Then we define `A_d`, `B_d`, and `C_d` of type `tensor_gpu`

```Fortran
    ! Tensors on the GPU
    type(tensor_gpu) :: A_d, B_d, C_d
```

#### Allocate memory

Memory allocations are performed using the `malloc` type procedure.

```Fortran
    ! Allocate memory for tensors, 
    ! see tensor_hip.f90 for 
    ! definition of generic procedures 
    call A_d%malloc(int(sizeof(A_h), c_size_t))
    call B_d%malloc(int(sizeof(B_h), c_size_t))
    call C_d%malloc(int(sizeof(C_h), c_size_t))
```

#### Copy memory from the host

Copying memory from the host is performed using the generic `copy_from` procedure. Since we have a procedure `copy_from_host_c_float_2` that handles 2D Fortran pointers, this call will be routed to that procedure.  

```Fortran
    call A_d%copy_from(A_h)
    call B_d%copy_from(B_h)
```

#### Kernel arguments

When launching the kernel we just pass in the `mem` field of the tensor type.

```Fortran
    ! Call the C function that launches the kernel
    call launch_kernel_hip( &
        A_d%mem, &
        B_d%mem, &
        C_d%mem, &
        int(M, c_int), &
        int(N, c_int) &
    )
```

#### Copy memory to the host

The copy back from `C_d` to `C_h` is accomplished with the `copy_to` generic procedure.

```Fortran
    ! Copy memory from the device to the host
    call C_d%copy_to(C_h)
```

#### Resource cleanup

When it comes time to clean up the memory we call the `free` procedure.

```Fortran
    ! Free tensors on the device
    ! this step is not necessary because 
    ! the tensor type has a destructor
    ! that is called when the tensor is out of scope
    call A_d%free
    call B_d%free
    call C_d%free
```

This step is not strictly necessary, as the destructor is automatically called when the tensors go out of scope. Then at cleanup we reset compute devices as per normal.

```Fortran
    ! Make sure all resources on the device are released
    call reset_device
```

<address>
Written by Dr. Toby Potter of <a href="https://www.pelagos-consulting.com">Pelagos Consulting and Education</a> and Dr. Joe Schoonover from <a href="https://www.fluidnumerics.com">Fluid Numerics</a>. All trademarks mentioned in this page are the property of their prospective owners.
</address> 